<a href="https://colab.research.google.com/github/tilly963963/pytorch_nlp/blob/main/gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.cnblogs.com/wwj99/p/12503545.html

In [1]:
import random

def select_top_k(predictions, k=10):
    predicted_index = random.choice(
        predictions[0, -1, :].sort(descending=True)[1][:10]).item()
    print("predicted_index=",predicted_index)
    return predicted_index

In [2]:
!pip install pytorch_transformers==1.0  # 安装 PyTorch-Transformers

     |████████████████████████████████| 143kB 13.7MB/s 
     |████████████████████████████████| 133kB 23.0MB/s 
     |████████████████████████████████| 1.2MB 23.6MB/s 
     |████████████████████████████████| 7.6MB 38.4MB/s 
     |████████████████████████████████| 81kB 9.3MB/s 
ERROR: botocore 1.20.84 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [14]:
import torch
from pytorch_transformers import GPT2Tokenizer

import logging
logging.basicConfig(level=logging.INFO)

# 载入预训练模型的分词器
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# 使用 GPT2Tokenizer 对输入进行编码
text = "the ct images were reviewed by two radiologists who reached decisions by consensus. we included only patients who presented with abnormalities on ct and in whom the diagnosis had been confirmed by pathological examination of the surgical specimen (if the lesion was resected). the ct scans were assessed in order to identify the main findings and to map the distribution of the lesions (i.e., to determine whether the pulmonary involvement was unilateral or bilateral). results: the main ct finding was the combination"
indexed_tokens = tokenizer.encode(text)
tokens_tensor = torch.tensor([indexed_tokens])
tokens_tensor.shape
print(tokens_tensor)

INFO:pytorch_transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json from cache at /root/.cache/torch/pytorch_transformers/f2808208f9bec2320371a9f5f891c184ae0b674ef866b79c58177067d15732dd.1512018be4ba4e8726e41b9145129dc30651ea4fec86aa61f4b9f40bf94eac71
INFO:pytorch_transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt from cache at /root/.cache/torch/pytorch_transformers/d629f792e430b3c76a1291bb2766b0a047e36fae0588f9dbc1ae51decdff691b.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda


tensor([[ 1169,   269,    83,  4263,   547, 11765,   416,   734, 19772,  9251,
           508,  4251,  5370,   416, 11529,    13,   356,  3017,   691,  3871,
           508,  5545,   351, 34615,   319,   269,    83,   290,   287,  4150,
           262, 13669,   550,   587,  4999,   416, 42306, 12452,   286,   262,
         21998, 31674,   357,   361,   262, 10287,   295,   373,  1599,   310,
           276,   737,   262,   269,    83, 23824,   547, 15276,   287,  1502,
           284,  5911,   262,  1388,  6373,   290,   284,  3975,   262,  6082,
           286,   262, 35258,   357,    72,    13,    68,  1539,   284,  5004,
          1771,   262, 45105,  9750,   373, 37095,   393, 24537,   737,  2482,
            25,   262,  1388,   269,    83,  4917,   373,   262,  6087]])


In [15]:
from pytorch_transformers import GPT2LMHeadModel

# 读取 GPT-2 预训练模型
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()

total_predicted_text = text
n = 100  # 预测过程的循环次数
for _ in range(n):
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    predicted_index = select_top_k(predictions, k=10)
    predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])
    total_predicted_text += tokenizer.decode(predicted_index)

    if '<|endoftext|>' in total_predicted_text:
        # 如果出现文本结束标志，就结束文本生成
        break

    indexed_tokens += [predicted_index]
    tokens_tensor = torch.tensor([indexed_tokens])

print(total_predicted_text)

INFO:pytorch_transformers.modeling_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-config.json from cache at /root/.cache/torch/pytorch_transformers/4be02c5697d91738003fb1685c9872f284166aa32e061576bbe6aaeb95649fcf.db13c9bc9c7bdd738ec89e069621d88e05dc670366092d809a9cbcac6798e24e
INFO:pytorch_transformers.modeling_utils:Model config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "finetuning_task": null,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 12,
  "n_positions": 1024,
  "num_labels": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use

the ct images were reviewed by two radiologists who reached decisions by consensus. we included only patients who presented with abnormalities on ct and in whom the diagnosis had been confirmed by pathological examination of the surgical specimen (if the lesion was resected). the ct scans were assessed in order to identify the main findings and to map the distribution of the lesions (i.e., to determine whether the pulmonary involvement was unilateral or bilateral). results: the main ct finding was the combination between bilateral hemorrhage with the pulmonary lesions as the dominant site of disease was associated strongly with hemorrhages with greater frequency compared With this study only 2 out 3 lesions were associated significantly with a bilateral pulmonary pathology and only 6 were statistically significantly significant and the rest did show some overlap. we found significant associations (p value < 10-6 and 0%, p >.005 in Fig 5) among the 3 sites of disease where hemorrhagic e